# Inspect decaying trajectories

In [1]:
import numpy as np
import os
import pickle

In [2]:
%matplotlib qt
import matplotlib.pyplot as plt

In [3]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot

Select folder where decay trajectories (.pkl files) are located and location of rest file (containing equilibrium position)

In [4]:
decayData_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic/north"

robot_dir = "../../../soft-robot-control/examples/trunk"
rest_file = os.path.join(robot_dir, 'rest_qv.pkl')

output_node = 51 # 'all' # 1354 # 
t_in = 1
t_out = 5

Load all trajectories

In [18]:
Data = {}
Data['oData'] = utils.import_pos_data(decayData_dir, rest_file, output_node, t_in, t_out, return_velocity=True)

In [23]:
print(Data['oData'][0][0].shape)
print(Data['oData'][0][1].shape)
nTraj = len(Data['oData'])
print(f"nTraj: {nTraj}")

(400,)
(6, 400)
nTraj: 40


In [20]:
with open(os.path.join(decayData_dir, "inputs.pkl"), "rb") as f:
    inputs = pickle.load(f)
inputs

[array([800.,   0.,   0., 800., 800.,   0.,   0.,   0.]),
 array([  0.,   0., 800.,   0., 800., 800.,   0., 800.]),
 array([800., 800.,   0., 800.,   0.,   0.,   0., 800.]),
 array([800.,   0.,   0., 800., 800.,   0.,   0., 800.]),
 array([800., 800., 800., 800.,   0.,   0., 800., 800.]),
 array([  0., 800., 800., 800., 800.,   0., 800.,   0.]),
 array([800., 800.,   0.,   0.,   0.,   0.,   0., 800.]),
 array([  0.,   0.,   0.,   0., 800.,   0., 800.,   0.]),
 array([800.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]),
 array([800.,   0., 800., 800., 800.,   0.,   0., 800.]),
 array([800.,   0., 800., 800.,   0., 800., 800.,   0.]),
 array([800.,   0.,   0., 800., 800.,   0., 800.,   0.]),
 array([800., 800.,   0.,   0., 800., 800., 800., 800.]),
 array([  0.,   0., 800., 800.,   0., 800.,   0.,   0.]),
 array([800., 800.,   0.,   0., 800.,   0., 800.,   0.]),
 array([  0., 800., 800.,   0., 800., 800.,   0., 800.]),
 array([800., 800., 800., 800., 800.,   0.,   0., 800.]),
 array([800., 

Check that all the initial conditions differ

In [8]:
# equal_trajectories = []
# for i in range(len(Data['oData'])):
#     equal_count = 0
#     for j in range(i, len(Data['oData'])):
#         if i == j:
#             continue
#         equal = np.allclose(Data['oData'][i][1][:, 0], Data['oData'][j][1][:, 0], rtol=0, atol=0.0000001)
#         if equal:
#             equal_count += 1
#             equal_trajectories.append((i, j))
#     if equal_count > 0:
#         print(f"something wrong with trajectory {i}", equal_count)
# print(equal_trajectories)

Plot decay data in (x, y, z)

In [21]:
# outdofs = [0, 1, 2]
plt.close('all')
# plot trajectories in 3D [x, y, z] space
plot.traj_3D(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'])
# plot evolution of x, y and z in time, separately in 3 subplots
highlight_traj = []
plot.traj_xyz(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'],
             highlight_idx=highlight_traj)
# plot velocities
plot.traj_xyz(Data,
             xyz_idx=[('oData', 3), ('oData', 4), ('oData', 5)],
             xyz_names=[r'$x$', r'$y$', r'$z$'],
             highlight_idx=highlight_traj)

Determine equilibrium position under given pre-tensioning

In [32]:
# print(Data['oData'][0][1][:3, -1])
z_eq = np.mean([Data['oData'][i][1][:3, -1] for i in range(nTraj)], axis=0)
qv_eq = np.hstack((z_eq, np.zeros(3)))
print(qv_eq)

[ 0.53263634 35.64735958 -6.36639933  0.          0.          0.        ]


Save equilibrium position into info.yaml dict

In [36]:
import yaml

with open(os.path.join(decayData_dir, "info.yaml"), "r") as f:
    info = yaml.safe_load(f)
info['qv_eq'] = qv_eq.astype(float).tolist()
with open(os.path.join(decayData_dir, "info.yaml"), "w") as f:
    yaml.dump(info, f)

Save data to .csv (one file per trajectory)

In [12]:
# from tqdm import tqdm

# for i in tqdm(range(1)): # range(len(Data['oData']))[0:1]:
#     oData = utils.import_pos_data(decayData_dir, rest_file, output_node, t_in, t_out, return_velocity=True, traj_index=i)
#     np.savetxt(f"/media/jonas/Backup Plus/jonas_soft_robot_data/data_roshan/decayTraj_pre-tensioned_{i:02d}.csv", oData[1], fmt='%.5e')

100%|██████████| 1/1 [00:23<00:00, 23.53s/it]


Save time array (only once, as it is the same for all trajectories)

In [ ]:
# oData = utils.import_pos_data(decayData_dir, rest_file, output_node, t_in, t_out, return_velocity=False, traj_index=0)
# np.savetxt(f"/media/jonas/Backup Plus/jonas_soft_robot_data/data_roshan/pre-tensioned_t.csv", oData[0], fmt='%.5e')